# Stochastic Series Expansion (SSE) on spin-1/2 Heisenberg antiferromagnet

This is an simple code adopted from [Prof. Anders W. Sandvik](https://physics.bu.edu/~sandvik/) for simulating the Heisenberg model on square lattice. To understand the following content in jupyter, we assume students understand the basic in Monte Carlo simulation and basic quantum statistical mechanics. 

### Background knowledge in Statistical mechanices

Denote $\psi$ as one of the possible configuration of the model. The average of certain function $f$ and partition funciton are as follows,

$$ \langle f \rangle = \frac{1}{Z} \sum f(\psi) W(\psi) $$
$$ Z = \sum_{\{ \psi \}} W(\psi) $$

where $W(\psi)$ is the weight of the configuration and $\{ \psi \}$ denote the set of all possible configurations of the model. 

As the model follows Maxwell-Boltzmann distribution, $W(\psi) = e^{- \beta E(\psi)}$, where $\beta = \frac{1}{T}$ is the inverse of temperature. 

Then, by consider the Taylor expansion of the $e^{- \beta E(\psi)}$ in the average expression,

$$e^{- \beta E(\psi)} = \sum_{n}^{\infin} \frac{(- \beta E(\psi) )^n}{n!}$$

we can rewrite the $\langle f \rangle$ and $Z$ as, 

$$ \langle f \rangle = \frac{1}{Z} \sum_{n}^{\infin} \sum_{\{ \psi \} } f(\psi)  \frac{(- \beta E(\psi) )^n}{n!} $$

$$ Z = \sum_{n}^{\infin} \sum_{\{ \psi \} } \frac{(- \beta E(\psi) )^n}{n!} $$ 

**(The following paragraph might need to be rewritten)**

We are now sampling not only in the space $\{ \psi \}$, but in the space $\{ (\psi,n) \}$, which includes an extra dimension $n$ (the expansion order n) for us to visit in Monte Carlo simulation. 

With this modifcation, we need to guarantee that the terms $ \frac{(- \beta E(\psi) )^n }{n!} $ need to be positive (i.e. since **weight or probability cannot be a negative number**.) 

Therefore, we define a new weight to be used for sampling in this configuration space as, 

$$W(\psi, n) = \frac{\beta^n [\epsilon - E(\psi) ]^n}{n!}$$

we can choose a suitable constant $\epsilon$ so that $\epsilon - E(\psi) > 0$

Define $H(\psi) = \epsilon - E(\psi)$. With this defintion, the weight $W(\psi, n)$ can be rewritten as, 

$$W(\psi, n) = \frac{\beta^n [H(\psi)]^n}{n!}$$

The expectation of $\langle H(\psi) \rangle$ and partition function $Z$ can then be written as,

$$\langle H \rangle = \frac{1}{Z} \sum_{n}^{\infin} \sum_{ \{ \psi \} } H(\psi) W(\psi,n)  $$

$$Z = \sum_{n}^{\infin} \sum_{\{ \psi \} } W(\psi,n) $$




|


Below are then a bit tricky **since $H$ can be written as 1 of the term of $W(\psi,1)$ and $\beta$**.
 
Consider only the terms in the nominator of $\langle H \rangle$ expression,

$$\sum_{n}^{\infin} \sum_{ \{ \psi \} } H(\psi) W(\psi,n)$$

\begin{align}
    \sum_{n}^{\infin} \sum_{ \{ \psi \} } H(\psi) W(\psi,n) &= \sum_{n}^{\infin} \sum_{ \{ \psi \} } W(\psi,1) W(\psi,n) \\
    &= \sum_{n}^{\infin} \sum_{ \{ \psi \} } \frac{\beta [H(\psi)]}{1} \frac{\beta^n [H(\psi)]^n}{n!} \\
    &= \sum_{n}^{\infin} \sum_{ \{ \psi \} } \frac{n+1}{\beta} \frac{\beta^{n+1} [H(\psi)]^{n+1}}{(n+1)!} \\
    &= \sum_{n}^{\infin} \sum_{ \{ \psi \} } \frac{n+1}{\beta} W(\psi, n+1) \\
    &= \frac{1}{\beta} \sum_{m-1}^{\infin} \sum_{ \{ \psi \} } m W(\psi, m)  
    \end{align} 

With the result above, the $\langle H \rangle$ can be written as,

$$\langle H \rangle = \frac{1}{\beta} \left( \frac{1}{Z}  \sum_{n-1}^{\infin} \sum_{ \{ \psi \} } n W(\psi, n) \right) = \frac{1}{\beta} \langle n \rangle_W$$

(Why can we arrive the last result? By comparing the 2nd last expression and the standard average formula of $\langle f \rangle$, you will know it.)

And, 

\begin{align}
    \langle H \rangle &= \frac{1}{\beta} \langle m \rangle_{W} \\
    \epsilon - E &= \frac{1}{\beta} \langle m \rangle_{W} \\
    E &= \epsilon - \langle m \rangle_{W}
    \end{align} 

This result is impressive since it means that, **if we want to know the energy of the model, we just need to keep track of the expansion order when we do the Monte Carlo simulation**. 


Starting from the Taylor expansion  $ e^ {-\beta H} $ = $ \sum _ {i=1}^ {\infty } $ $ \frac {(-\beta )^ {n}}{n!} $ $ H^ {n} $, the partition function is then written as:
$$
Z_ {SSE} = \sum _ {n=0}^ {\infty }  \frac {\beta ^ {n}}{n!} \sum _ {\{\alpha\} _n}{\langle\alpha _ {0}|H|\alpha _ {n-1}}\rangle\cdots  \langle  \alpha _ {2}  |H  | \alpha _ {1} \rangle \langle  \alpha _ {1}  |H |  \alpha _ {0}\rangle ,
$$
where $\{\alpha\}_n$ denotes a complete set of states.

## Code

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt


In [ ]:
def init_config(nn):
    global spin
    spin = [0]*nn

    for i in range(nn):
        spin[i] = 2*int(2*np.random.random())-1

    global mm
    mm = 20

    global opstring,nh
    opstring = [0]*mm
    nh = 0


In [ ]:
def makelattice():
    global nn,nb,lx,ly,bsites
    nn = lx * ly

    if ly!=1:
        nb = 2*nn
        bsites = np.array([[0]*nb,[0]*nb])

        for y1 in range(0,ly):
            for x1 in range(0,lx):
                s = 1+x1+y1*lx
                x2 = np.mod(x1+1,lx)
                y2 = y1
                bsites[0][s-1] = s
                bsites[1][s-1] = 1+x2+y2*lx
                x2 = x1
                y2 = np.mod(y1+1,ly)
                bsites[0][s+nn-1] = s
                bsites[1][s+nn-1] = 1+x2+y2*lx
    else:
        nn = lx
        nb = nn
        bsites = np.array([[0]*nb,[0]*nb])

        for x1 in range(lx):
            bsites[0][x1] = x1+1
            bsites[1][x1] = 1+x1+1

        bsites[1][lx-1] = 1


In [ ]:
def adjustcutoff():
    global mm,opstring
    mmnew = nh+nh//3
    if mmnew <= mm:
        return
    diff = mmnew-mm
    mm = mmnew
    opstring.extend([0]*(diff))

In [ ]:
def diagonalupdate():

    global spin,mm,nb,nh,aprob,dprob

    for i in range(mm):
        op = opstring[i]
        if op == 0:
            b = min(int(np.random.random()*nb)+1,nb)
            if spin[int(bsites[0][b-1])-1] != spin[int(bsites[1][b-1])-1]:
                if (aprob >= mm-nh) or (aprob >= np.random.random()*(mm-nh)):
                    opstring[i] = 2*b
                    nh += 1
        elif np.mod(op,2) == 0:
            p = dprob*(mm-nh+1)
            if (p >= 1 ) or (p>=np.random.random()):
                opstring[i] = 0
                nh -= 1
        else:
            b = op//2
            spin[int(bsites[0][b-1])-1] = -spin[int(bsites[0][b-1])-1]
            spin[int(bsites[1][b-1])-1] = -spin[int(bsites[1][b-1])-1]



In [ ]:
def loopupdate():
    global vertexlist,frstspinop,lastspinop,opstring,bsites,spin,mm,nn

    frstspinop = [0]*nn
    lastspinop = [0]*nn
    vertexlist = [0]*(4*mm)

    for i in range(nn):
        frstspinop[i] = -1
        lastspinop[i] = -1

    for v0 in np.arange(0,4*mm,4):
        op = opstring[v0//4]
        if op != 0:
            b = op//2
            s1 = bsites[0][b-1]
            s2 = bsites[1][b-1]
            v1 = lastspinop[s1-1]
            v2 = lastspinop[s2-1]
            if v1 != -1:
                vertexlist[v1] = v0
                vertexlist[v0] = v1
            else:
                frstspinop[s1-1] = v0

            if v2!=-1:
                vertexlist[v2] = v0+1
                vertexlist[v0+1] = v2
            else:
                frstspinop[s2-1] = v0+1

            lastspinop[s1-1] = v0+2
            lastspinop[s2-1] = v0+3
        else:
            for i in np.arange(v0,v0+4):
                vertexlist[i] = 0

    for sn in np.arange(0,nn): #sn is the spin number from 0 to nn-1
        v1 = frstspinop[sn]
        if v1!=-1:
            v2 = lastspinop[sn]
            vertexlist[v2] = v1
            vertexlist[v1] = v2

    for v0 in np.arange(0,4*mm,2):
        if vertexlist[v0]<1:
            continue
        v1 = v0
        if np.random.random()<0.5:
            while True:
                opstring[v1//4] = opstring[v1//4]^1
                vertexlist[v1] = -1
                v2 = v1^1
                v1 = vertexlist[v2]
                vertexlist[v2] = -1
                if v1==v0:
                    break
        else:
            while True:
                vertexlist[v1]=0
                v2 = v1^1
                v1 = vertexlist[v2]
                vertexlist[v2] = 0
                if v1==v0:
                    break
    for i in range(nn):
        if frstspinop[i]!=-1:
            if vertexlist[int(frstspinop[i])]==-1:
                spin[i] = -spin[i]
        else:
            if np.random.random()<0.5:
                spin[i] = -spin[i]





In [ ]:
def measureobservables():
    global enrg1,enrg2,amag1,amag2,am1,am2,ax1,opstring,bsites,spin,mm,nn
    am =0
    for i in range(nn):
        am = am+spin[i]*(-1)**(np.mod(i,lx)+i//lx)
    am = am/2
    am1 = 0.0
    am2 = 0.0
    ax1 = 0.0

    for i in range(mm):
        op = opstring[i]
        if op ==0:
            continue
        elif np.mod(op,2) ==1:
            b = op//2
            s1 = bsites[0][b-1]
            s2 = bsites[1][b-1]
            spin[s1-1] = -spin[s1-1]
            spin[s2-1] = -spin[s2-1]
            am = am+2*spin[s1-1]*(-1)**(np.mod(s1-1,lx)+(s1-1)//lx)
        ax1 = ax1+float(am)
        am1 = am1+float(abs(am))
        am2 = am2+float(am)**2

    if nh!=0:
        ax1 = (ax1**2+am2)/(float(nh)*float(nh+1))
        am1 = am1/nh
        am2 = am2/nh
    else:
        am1 = float(abs(am))
        am2 = float(am)**2
        ax1 = am2



In [ ]:
#main
start = time.time()

lx,ly = 8,1
beta=16
nbin=10

E_lst = []
E_err = []

makelattice()
init_config(nn)

aprob = 0.5*beta*nb
dprob = 1.0/(0.5*beta*nb)

nh_lst = []
nh_mc = []
dis = []

eqstep = 2000
mcstep = 20000
nbin = 10
bin_step = mcstep//nbin

for i in range(eqstep):
    diagonalupdate()
    loopupdate()
    adjustcutoff()


for i in range(mcstep):
    diagonalupdate()
    loopupdate()
    nh_mc.append(nh)
    dis.append(nh)
    if (i+1)%(bin_step) == 0:
        print("bin:",(i+1)//bin_step)
        nh_lst.append(np.mean(nh_mc))
        nh_mc = []


E = -(np.mean(nh_lst)/(beta*nn)-nb/(4*nn))
err = np.std(nh_lst)/(beta*nn)/np.sqrt(nbin)
E_lst.append(E)
E_err.append(err)

end = time.time()
print("Running time:", end-start)
print("E:",E_lst[0])
print("err:",E_err[0])



bin: 1
bin: 2
bin: 3
bin: 4
bin: 5
bin: 6
bin: 7
bin: 8
bin: 9
bin: 10
Running time: 29.01471185684204
E: -0.45647265625
err: 0.0008489840550806885
